<a href="https://colab.research.google.com/github/jpnoxon/jpnoxon_64061_MachineLearning/blob/main/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is for Advanced Machine Learning, Assignment 2


### I'll start by downloading the data, following along with the book's example


In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"jasonnoxon","key":"d30c331476fa48f0b7e0d3be15227973"}'}

In [4]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -c dogs-vs-cats

100% 541M/543M [00:03<00:00, 150MB/s]
100% 543M/543M [00:03<00:00, 150MB/s]
  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 85.7MB/s]
 98% 266M/271M [00:01<00:00, 164MB/s]
100% 271M/271M [00:01<00:00, 153MB/s]


In [6]:
!unzip -qq train.zip

Per assingment instructions, leaving the training set at 1000, validation at 500 and test at 500 - Jason N


In [7]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2000)

### Building the model

To Improve the model provided in the text, and to reduce overfitting, I'm implementing separable convolution layers
Separable convolution layers assume different channels are highly independent while intermediate activations are highly correlated 

I'm applying a dropout layers to reduce overfitting as well. 

Lastly, I'm augmenting the data inputs to reduce overfitting

-Jason N

In [93]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(inputs)
x = layers.BatchNormalization()(x)
x = layers.SeparableConv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.BatchNormalization()(x)
x = layers.SeparableConv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [95]:
model.summary()

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        [(None, 180, 180, 3)]     0         
_________________________________________________________________
rescaling_28 (Rescaling)     (None, 180, 180, 3)       0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 180, 180, 3)       12        
_________________________________________________________________
separable_conv2d_70 (Separab (None, 178, 178, 128)     539       
_________________________________________________________________
max_pooling2d_57 (MaxPooling (None, 89, 89, 128)       0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 89, 89, 128)       512       
_________________________________________________________________
separable_conv2d_71 (Separab (None, 87, 87, 256)       341

chaning the optimizer to adam from rmsprop - Jason N

In [96]:
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

### Data preprocessing

I'm changing the batch size to 64 for each data set - Jason N

---



In [97]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=64)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=64)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=64)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [98]:
import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

In [99]:
for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

(16,)
(16,)
(16,)


In [100]:
batched_dataset = dataset.batch(32)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

(32, 16)
(32, 16)
(32, 16)


In [101]:
reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i >= 2:
        break

(4, 4)
(4, 4)
(4, 4)


**Displaying the shapes of the data and labels yielded by the `Dataset`**

In [102]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (64, 180, 180, 3)
labels batch shape: (64,)


Using 10 epochs for the model due to GPU constaints - Jason N

In [103]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/10
32/32 [==============================] - 253s 8s/step - loss: 1.1670 - accuracy: 0.5260 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
32/32 [==============================] - 250s 8s/step - loss: 0.5793 - accuracy: 0.7115 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
32/32 [==============================] - 251s 8s/step - loss: 0.5029 - accuracy: 0.7815 - val_loss: 0.6931 - val_accuracy: 0.5010
Epoch 4/10
32/32 [==============================] - 252s 8s/step - loss: 0.4072 - accuracy: 0.8270 - val_loss: 0.6928 - val_accuracy: 0.5250
Epoch 5/10
32/32 [==============================] - 252s 8s/step - loss: 0.2899 - accuracy: 0.9035 - val_loss: 0.6917 - val_accuracy: 0.5380
Epoch 6/10
32/32 [==============================] - 252s 8s/step - loss: 0.1991 - accuracy: 0.9390 - val_loss: 0.6856 - val_accuracy: 0.5990
Epoch 7/10
32/32 [==============================] - 253s 8s/step - loss: 0.1116 - accuracy: 0.9715 - val_loss: 0.6680 - val_accuracy: 0.6380
Epoch 8/10
32

The validation accuracy and validation loss is dropping with each epoch which suggests the model isn't overfitting
With more epochs, the model would continue improving - Jason N

**Evaluating the model on the test set**

In [104]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

16/16 [==============================] - 24s 1s/step - loss: 0.6342 - accuracy: 0.6470
Test accuracy: 0.647


Model accuracy is right inline with validation accuracy of model
prediction is about 2/3 being predicted correctly
More epochs would increase model performance since it hasn't overfit yet
Ideally, with the right computing power, I'd set the number of epochs and graph validation accuracy to see if the model has overfit, then set the optimal number of epochs.

Increasing the training dataset to 1500 per assignment instructions and re-training the model - Jason N.

In [8]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname,
                            dst=dir / fname)

make_subset("train", start_index=0, end_index=1500)
make_subset("validation", start_index=1500, end_index=2000)
make_subset("test", start_index=2000, end_index=2500)

In [120]:


from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(inputs)
x = layers.BatchNormalization()(x)
x = layers.SeparableConv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.BatchNormalization()(x)
x = layers.SeparableConv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(180, 180),
    batch_size=64)
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(180, 180),
    batch_size=64)
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(180, 180),
    batch_size=64)


import numpy as np
import tensorflow as tf
random_numbers = np.random.normal(size=(1000, 16))
dataset = tf.data.Dataset.from_tensor_slices(random_numbers)

for i, element in enumerate(dataset):
    print(element.shape)
    if i >= 2:
        break

batched_dataset = dataset.batch(64)
for i, element in enumerate(batched_dataset):
    print(element.shape)
    if i >= 2:
        break

reshaped_dataset = dataset.map(lambda x: tf.reshape(x, (4, 4)))
for i, element in enumerate(reshaped_dataset):
    print(element.shape)
    if i >= 2:
        break

for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="convnet_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks)

Found 3000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
(16,)
(16,)
(16,)
(64, 16)
(64, 16)
(64, 16)
(4, 4)
(4, 4)
(4, 4)
data batch shape: (64, 180, 180, 3)
labels batch shape: (64,)
Epoch 1/10
47/47 [==============================] - 391s 8s/step - loss: 1.3371 - accuracy: 0.5307 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/10
47/47 [==============================] - 387s 8s/step - loss: 0.5786 - accuracy: 0.7070 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/10
47/47 [==============================] - 389s 8s/step - loss: 0.5046 - accuracy: 0.7560 - val_loss: 0.6930 - val_accuracy: 0.5015
Epoch 4/10
47/47 [==============================] - 389s 8s/step - loss: 0.4100 - accuracy: 0.8210 - val_loss: 0.6902 - val_accuracy: 0.5190
Epoch 5/10
47/47 [==============================] - 390s 8s/step - loss: 0.3312 - accuracy: 0.8630 - val_loss: 0.6758 - val_accuracy: 0.6400
Epoch 6/10
47/47 [==========================

In [121]:
test_model = keras.models.load_model("convnet_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

32/32 [==============================] - 51s 2s/step - loss: 0.6801 - accuracy: 0.6940
Test accuracy: 0.694


Increasing the size of the training sample drastically increased validation accuracy from 68% to 84% and reduced validation loss withthe same number of epochs, however,  accuracy on the test data set only increased from 64% to 69%.

Using a pre-trained convolution model per assignment instructions

In [ ]:

conv_base  = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False)
conv_base.trainable = False

**Printing the list of trainable weights before and after freezing**

**Adding a data augmentation stage and a classifier to the convolutional base**

In [13]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

NameError: ignored

Limiting the model to 10 epochs to compare it with the model trained from scratch.  Keeping the data at 

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="feature_extraction_with_data_augmentation.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks)

Epoch 1/10
47/47 [==============================] - ETA: 0s - loss: 26.0873 - accuracy: 0.8960 

**Evaluating the model on the test set**

In [ ]:
test_model = keras.models.load_model(
    "feature_extraction_with_data_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In order to fine tune a pre-trained model, we have to adjust the learning rate of the model.  The learning rate has an impact on finding local minimums and maximums.  large adjustments to the learning rate will have drastic impacts on the model.  

In [ ]:
model.compile(loss="binary_crossentropy",
              optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="fine_tuning.keras",
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
model = keras.models.load_model("fine_tuning.keras")
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")